In [1]:
import logging
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import awkward as ak
import uproot
import keras
import keras.layers as layers
from DeepSetNeuralNetArchitecture import Mean_Squared_Error
from keras import regularizers
from Sum import Sum
import matplotlib.pyplot as plt
from hffrag import fixedbinning
from hffrag import binneddensity
from numpy.lib.recfunctions import structured_to_unstructured
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-01-16 00:35:53.699046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 00:35:53.844050: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-16 00:35:53.848329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 00:35:53.848344: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

number of jets to train on:
141329


/home/physics/phujdj/DeepLearningParticlePhysics/hffrag.py:136: RuntimeWarning: overflow encountered in sinh
  pzs = numpy.where(mask, pts, pts * numpy.sinh(etas))


In [2]:
# Find the associated tracks for each jet
def Match_Tracks(jets, tracks):
    """Used to determine if a set of tracks belong to a particular set of jets"""

    jet_eta = jets["AnalysisAntiKt4TruthJets_eta"]
    jet_phi = jets["AnalysisAntiKt4TruthJets_phi"] 

    tracks_eta = tracks["AnalysisTracks_eta"]
    tracks_phi = tracks["AnalysisTracks_phi"]

    delta_etas = jet_eta - tracks_eta
    delta_phis = np.abs(jet_phi - tracks_phi)

    # Map the phis from a cyclical period onto a linear relation
    ak.where(delta_phis > np.pi, delta_phis - np.pi, delta_phis)

    # Returns a list of true and false, determining which tracks belong to those jets.
    return np.sqrt(delta_phis**2 + delta_etas**2) < 0.4 

In [3]:
def pt_eta_phi_2_px_py_pz_tracks(pt_eta_phi, MASKVAL=-999):
    """Converts the cylindrical polar coordinates to cartesian coordinates for jets"""

    # Seperate the pts, etas and phis
    pts = pt_eta_phi[:, :, 0:1]
    etas = pt_eta_phi[:, :, 1:2]
    phis = pt_eta_phi[:, :, 2:3]

    # Convert from polar to cartesian
    # Transforms only the non masked values from cylindrical to cartesian coordinates. Mask values are left unchanged.
    mask1 = pts == MASKVAL 
    mask2 = phis == MASKVAL
    mask3 = etas == MASKVAL
    pxs = np.where(mask1 | mask2, pts, pts * np.cos(phis)) 
    pys = np.where(mask1 | mask2, pts, pts * np.sin(phis))
    pzs = np.where(mask1 | mask3, pts, pts * np.sinh(etas))

    # Check to see if there are any infinities
    isinf = np.isinf(pzs)

    if np.any(isinf):
        print("Infinities in eta detected!")
        print(etas[isinf])
        raise ValueError("Infinity from sinh(eta) has been detected")

    # Returns the momentum vector in cartesian coordinates
    return np.concatenate([pxs, pys, pzs], axis=2)

In [4]:
def pt_eta_phi_2_px_py_pz_tracks(pt_eta_phi, MASKVAL=-999):
    """Converts the cylindrical polar coordinates to cartesian coordinates for jets"""

    # Seperate the pts, etas and phis
    pts = pt_eta_phi[:, :, 0:1]
    etas = pt_eta_phi[:, :, 1:2]
    phis = pt_eta_phi[:, :, 2:3]

    # Convert from polar to cartesian
    # Transforms only the non masked values from cylindrical to cartesian coordinates. Mask values are left unchanged.
    mask1 = pts == MASKVAL 
    mask2 = phis == MASKVAL
    mask3 = etas == MASKVAL
    pxs = np.where(mask1 | mask2, pts, pts * np.cos(phis)) 
    pys = np.where(mask1 | mask2, pts, pts * np.sin(phis))
    pzs = np.where(mask1 | mask3, pts, pts * np.sinh(etas))

    # Check to see if there are any infinities
    isinf = np.isinf(pzs)

    if np.any(isinf):
        print("Infinities in eta detected!")
        print(etas[isinf])
        raise ValueError("Infinity from sinh(eta) has been detected")

    # Returns the momentum vector in cartesian coordinates
    return np.concatenate([pxs, pys, pzs], axis=2)


In [5]:
def pt_eta_phi2_px_py_pz_predicted_tracks(predictions):
    #Obtain the pts,etas and phis
    pts = predictions[:,0:1]
    etas = predictions[:,1:2]
    phis = predictions[:,2:3]

    # Convert from polar to cartesian
    # Transforms only the non masked values from cylindrical to cartesian coordinates. Mask values are left unchanged.
    pxs =  pts * np.cos(phis)
    pys =  pts * np.sin(phis)
    pzs =  pts * np.sinh(etas)

    # Check to see if there are any infinities
    isinf = np.isinf(pzs)

    if np.any(isinf):
        print("Infinities in eta detected!")
        print(etas[isinf])
        raise ValueError("Infinity from sinh(eta) has been detected")

    # Returns the momentum vector in cartesian coordinates
    return np.concatenate([pxs, pys, pzs], axis=-1)

In [6]:
# Convert from cylindrical to cartesian coordinates
def pt_eta_phi_2_px_py_pz_jets(pt_eta_phi):
    """Converts the cylindrical polar coordinates to cartesian coordinates for jets"""

    # Seperate the pts, etas and phis
    pts = pt_eta_phi[:, 0:1]
    etas = pt_eta_phi[:, 1:2]
    phis = pt_eta_phi[:, 2:3]

    # Convert from polar to cartesian
    pxs = pts * np.cos(phis)
    pys = pts * np.sin(phis)
    pzs = pts * np.sinh(etas)

    # Check to see if there are any infinities
    isinf = np.isinf(pzs)

    if np.any(isinf):
        print("Infinities in eta detected!")
        print(etas[isinf])
        raise ValueError("Infinity from sinh(eta) has been detected")

    # Returns the momentum vector
    return np.concatenate([pxs, pys, pzs], axis=1)

In [7]:
def pad(x_values, maxsize, MASKVAL=-999):
    """
    Pads the inputs with nans to get to the maxsize
    """
    #Pad the non-regular arrays with null values until they are all of the same size. Then replace the nulls with MASVAL
    y_values = ak.fill_none(ak.pad_none(x_values, maxsize, axis=1, clip=True), MASKVAL)[:, :maxsize]
    return ak.to_regular(y_values, axis=1) #Return the regular arrays

In [8]:
def flatten(x_values, maxsize=-1, MASKVAL=-999):
    """"Pads the input to ensure they are all of regular size and then zips together result"""
    y_values = {} 
    for field in x_values.fields:
        z_values = x_values[field]
        if maxsize > 0:
            z_values = pad(z_values, maxsize, MASKVAL)
        y_values[field] = z_values

    return ak.zip(y_values)

In [9]:
def LogNormal_Loss_Function(true,mean_convariance_matrix):
    
    """A custom loss function designed to force the neural network 
    to return a prediction and associated uncertainty for target features"""

    #Identify the number of target features
    n_targets = np.shape(true)[1]

    #Allocate the first n outputs of the dense layer to represent the mean
    means = mean_convariance_matrix[:, :n_targets]

    #Allocate the second n outputs of the dense layer to represent the variances
    logvariances = mean_convariance_matrix[:, n_targets: 2* n_targets]

    #Allocate the last n outputs of th4e dense layer to represent the covariances
    logcovariances = mean_convariance_matrix[:, 2*n_targets:]


    #Calculate the logNormal loss
    sum_loss = 0
    for target in range(n_targets):
        sum_loss += (1/2)*keras.backend.log(2*np.pi) + logvariances[:,target] + ((true[:,target] - means[:,target])**2)/(2*keras.backend.exp(logvariances[:,target])**2)
    
    return sum_loss

In [10]:
# The data is being stored in a tree datastructure.
# We access the charm root using this command
tree = uproot.open("hffrag.root:CharmAnalysis")

In [11]:
# Initial parameters
MASKVAL = -999 # This value is introduced to ensure arrays are regular (Of the same size). They will be masked later by the network
MAXTRACKS = 32 # This value is the maximum number of tracks allowed per event
BATCHSIZE = 64 # This is the batch size of the mini batches used during training
EPOCHS = 100 # This is the default number of epochs for which the neural network will train providing that early stopping does not occur
MAXEVENTS = 1e20 #This is the maximum number of events that will the program will accept
LR = 1e-4 #This is the default learning rate

In [12]:
# Select the features we wish to study
track_features = ["AnalysisTracks_pt", "AnalysisTracks_eta", "AnalysisTracks_phi", "AnalysisTracks_z0sinTheta",
                  "AnalysisTracks_d0sig", "AnalysisTracks_d0", "AnalysisTracks_d0sigPV", "AnalysisTracks_d0PV"]
jet_features = ["AnalysisAntiKt4TruthJets_pt", "AnalysisAntiKt4TruthJets_eta", "AnalysisAntiKt4TruthJets_phi",
                "AnalysisAntiKt4TruthJets_ghostB_pt", "AnalysisAntiKt4TruthJets_ghostB_eta","AnalysisAntiKt4TruthJets_ghostB_phi"]

In [13]:
# Read in the dat from the root file
features = tree.arrays(jet_features+track_features, entry_stop=MAXEVENTS)

In [14]:
# Select the events of interest
events = features[ak.sum(
    features["AnalysisAntiKt4TruthJets_pt"] > 25000, axis=1) > 0]

In [15]:
# Displays the number of jets being trained on
jets = events[jet_features][:, 0]
print("The number of jets to train on is: ", len(jets))
print("The number of track features is: ",len(track_features))

The number of jets to train on is:  141329
The number of track features is:  8


In [16]:
# Select tracks from the events
tracks = events[track_features]

# Match the tracks to the jets
matchedtracks = tracks[Match_Tracks(jets, tracks)]

# Pad and Flatten the data
matchedtracks = flatten(matchedtracks, MAXTRACKS)

In [17]:
# Identify the the bottom jets and their associated tracks
bjets = ak.sum(jets["AnalysisAntiKt4TruthJets_ghostB_pt"] > 5000, axis=1) > 0
jets = jets[bjets]

# Obtain the pt, eta and phi of each b hadron jet
bhads_pt = jets["AnalysisAntiKt4TruthJets_ghostB_pt"][:, 0].to_numpy()
bhads_eta = jets["AnalysisAntiKt4TruthJets_ghostB_eta"][:,0].to_numpy()
bhads_phi = jets["AnalysisAntiKt4TruthJets_ghostB_phi"][:,0].to_numpy()

bhads = np.stack([bhads_pt,bhads_eta,bhads_phi],axis = -1) #Combine the momentum, eta and phi for each jet into one array

print("There are {} outputs".format(np.shape(bhads)[1])) # Display the number of target features the neural network will predict
matchedtracks = matchedtracks[bjets]
print("There are {} inputs".format(np.shape(matchedtracks)[1])) # Display the number of target features the neural network will use in it's ppredictions

There are 3 outputs
There are 32 inputs


In [18]:
print(np.shape(bhads)) #Check the shape of the neural network
print(np.shape(jet_features[:-1])) #Check for shape of the jet features
print(jets[jet_features[0]]) # Check the jets

(68143, 3)
(5,)
[1.48e+05, 1.04e+05, 1.16e+05, 4.03e+04, ... 8.14e+04, 9.83e+04, 1.45e+05, 9.11e+04]


In [19]:
# Transform the jet and tracks to unstructed data.
jets = structured_to_unstructured(jets[jet_features[:-3]])
matchedtracks = structured_to_unstructured(matchedtracks)
print(np.shape(jets))

(68143, 3)


In [20]:
#Check the matchtracks are the correct shape
print(matchedtracks[:, 0:1])
print(np.shape(matchedtracks[:, :, 3]))

[[[1.47e+04, 0.753, 1.14, 1.19, 75.5, ... -0.165, -0.51, -0.0283, -0.692, -0.038]]]
(68143, 32)


In [21]:
# Convert the coordinates of the b jets and tracks to cartesian coordinates
tracks_p = pt_eta_phi_2_px_py_pz_tracks(matchedtracks.to_numpy())
bhads = pt_eta_phi_2_px_py_pz_jets(bhads)

#Check the shape of the momenta of the tracks and the rest of the data is consistent
print(np.shape(tracks_p))
print(np.shape(matchedtracks[:, :, 3:]))

#Combine the momenta of the tracks with the rest of the track features to form the track dataset
tracks = np.concatenate([tracks_p,matchedtracks[:,:,3:].to_numpy()],axis = 2)

(68143, 32, 3)
(68143, 32, 5)


/tmp/ipykernel_21007/1222727277.py:16: RuntimeWarning: overflow encountered in sinh
  pzs = np.where(mask1 | mask3, pts, pts * np.sinh(etas))


In [22]:
#Check that this is all the correct shape
print(np.shape(tracks))
print(np.shape(bhads))
print(tracks[0,0])
print(bhads[0])

(68143, 32, 8)
(68143, 3)
[6.20926450e+03 1.33553447e+04 1.21693980e+04 1.18753994e+00
 7.55359192e+01 1.33110714e+00 8.57456207e+01 1.32391548e+00]
[ 48855.56531144 128363.19160447 124938.01790683]


In [23]:
Scaler = StandardScaler()
Num_events,Num_tracks,Num_features = np.shape(tracks)
tracks = np.reshape(tracks, newshape=(-1,Num_features))
tracks = Scaler.fit_transform(tracks)
tracks = np.reshape(tracks, newshape= (Num_events,Num_tracks,Num_features))
print(np.shape(tracks))
print(tracks[0,0,:])

(68143, 32, 8)
[0.26189855 0.38175348 0.10230412 1.42959932 1.58783932 1.42992229
 1.60955267 1.42990682]


In [24]:
# Split the data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(
    tracks, bhads, train_size=0.8, random_state=42)

In [25]:
class ParticleEmbbedder(tf.keras.layers.Layer):
    def __init__(self, track_layers,BATCH_SIZE, MASKVAL):
        super().__init__()
        self.track_layers = track_layers
        self.MASKVAL = MASKVAL
        self.BATCH_SIZE = BATCH_SIZE
        self.mask = tf.keras.layers.Masking(mask_value=MASKVAL)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(track_layers[0],activation = "elu")),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(track_layers[1],activation = "elu")),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(track_layers[2],activation = "elu")),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(track_layers[3],activation = "elu")),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(self.BATCH_SIZE, activation = "elu"))
        ])
        
    def call(self,x):
        outputs = self.mask(x)
        outputs = self.ffn(x)
        return outputs
    

In [26]:
class BhadEmbedder(tf.keras.layers.Layer):
    def __init__(self, d_model, MASKVAL):
        super().__init__()
        self.MASKVAL = MASKVAL
        self.d_model = d_model
        self.mask = tf.keras.layers.Masking(mask_value=MASKVAL)
        self.embedding = tf.keras.Sequential([
            tf.keras.layers.Dense(d_model/8),
            tf.keras.layers.Dense(d_model/4),
            tf.keras.layers.Dense(d_model/2),
            tf.keras.layers.Dense(d_model)
        ])
        
    def call(self,x):
        outputs = self.mask(x)
        outputs = self.embedding(outputs)
        return outputs
    

In [27]:
class Jet_Layer(keras.layers.Layer):
    def __init__(self, dff, n_targets,d_model, MASKVAL=-999):
        super().__init__()
        self.mask = tf.keras.layers.Masking()
        self.n_targets = n_targets
        self.d_model = d_model
        self.dff = dff
        self.ffn = tf.keras.Sequential([
        layers.Dense(self.dff, activation = "elu"),
        layers.Dense(self.d_model),
        layers.Dense(self.n_targets+self.n_targets*(self.n_targets+1)//2),
        ])

    def call(self,x):
        x = self.mask(x)
        x = self.ffn(x)
        return x

In [28]:
print(X_train.shape)
print(len(y_train))

(54514, 32, 8)
54514


In [29]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self,MASKVAL, **kwargs):
        super().__init__()
        self.masking = tf.keras.layers.Masking(MASKVAL)
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [30]:
class GlobalSelfAttention(BaseAttention):
    def call(self,x):
        attn_output = self.mha(
            query = x,
            value = x,
            key = x,
        )
        x = self.masking(x)
        x  = self.add([x,attn_output])
        x = self.layernorm(x)
        return x

In [31]:
class CrossAttention(BaseAttention):
    def call(self,x,context):
        attn_output, attn_scores = self.mha(query = x, key = context, 
        value = context,
        return_attention_scores = True)
    
        self.last_attn_scores = attn_scores
        
        x = self.masking(x)
        x = self.add([x,attn_output])
        x = self.layernorm(x)

        return x

In [32]:
class CausalSelfAttention(BaseAttention):
    def call(self,x):
        attn_output = self.mha(
            query = x,
            value = x,
            key = x,
            use_causal_mask = True)
            
        x = self.masking(x)
        x = self.add([x,attn_output])
        x = self.layernorm(x)
        return x

In [33]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self,d_model, dff, dropout = 0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff,activation = 'gelu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self,x):
        x = self.add([x,self.seq(x)])
        x = self.layer_norm(x)
        return x

In [34]:
class ParticleAttentionBlock(tf.keras.layers.Layer):
    def __init__(self, *, d_model, num_heads, dff, MASKVAL = -999, dropout_rate = 0.1):
        super().__init__()
    
        self.self_attention = GlobalSelfAttention(
            MASKVAL = MASKVAL,
            num_heads = num_heads,
            key_dim = d_model,
            dropout = dropout_rate
        )
    
        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

In [35]:
class BhadAttentionBlock(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1,
               MASKVAL = MASKVAL):
    super(BhadAttentionBlock, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate,
        MASKVAL= MASKVAL)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate,
        MASKVAL=MASKVAL)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)
    return x

In [36]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, *, track_layers,num_layers, d_model, num_heads, dff,MASKVAL = -999, dropout_rate = 0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.ParticleEmbbedder = ParticleEmbbedder(track_layers,d_model,MASKVAL)
        
        self.encoder_layers = [
            ParticleAttentionBlock(
            d_model=d_model,
            MASKVAL = MASKVAL,
            num_heads = num_heads,
            dff = dff,
            dropout_rate = dropout_rate
            )
        for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x):
        x = self.ParticleEmbbedder(x)

        x = self.dropout(x)

        for increment in range(self.num_layers):
            x = self.encoder_layers[increment](x)

        return x

In [37]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, n_targets, MASKVAL = MASKVAL,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.BhadEmbedder = BhadEmbedder( d_model = d_model, MASKVAL=MASKVAL)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        BhadAttentionBlock(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    x = self.BhadEmbedder(x)
    x = self.dropout(x)

    for increment in range(self.num_layers):
      x  = self.dec_layers[increment](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    return Sum()(x)

In [38]:
class hffragTransformer(tf.keras.Model):
    def __init__(self, *,track_layers, num_layers, d_model, num_heads, dff, n_targets, MASKVAL = MASKVAL, dropout = 0.1):
        super().__init__()

        self.encoder = Encoder(track_layers = track_layers, d_model = d_model, num_layers= num_layers, num_heads=num_heads,dff = dff, MASKVAL= MASKVAL, dropout_rate = dropout)

        self.decoder = Decoder(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff = dff, n_targets=n_targets, MASKVAL=MASKVAL, dropout_rate=dropout)

        self.jet_layer = Jet_Layer(n_targets= n_targets, dff = dff, d_model= d_model)
        
    def call(self, inputs):
        context, x = inputs

        context = self.encoder(context)

        x = self.decoder(x, context)

        outputs = self.jet_layer(x)

        return outputs

In [39]:
track_layers = [32,32,32,32,32]
BATCHSIZE = 128
sample_particle_embedder = ParticleEmbbedder(track_layers,BATCHSIZE,MASKVAL)
output = sample_particle_embedder(X_train)
print(output.shape)

2023-01-16 00:37:48.204131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-16 00:37:48.204163: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-16 00:37:48.204183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vonneumann.csc.warwick.ac.uk): /proc/driver/nvidia/version does not exist
2023-01-16 00:37:48.204878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(54514, 32, 128)


In [40]:
sample_Bhad = BhadEmbedder( d_model = 64, MASKVAL=MASKVAL)
sample = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))
print(sample.shape)
sample_2 = sample_Bhad(y_train)
print(sample_2.shape)

(54514, 3, 1)
(54514, 64)


In [41]:
sample_gsa = GlobalSelfAttention(MASKVAL = -999, num_heads = 8, key_dim = 3)

print(np.shape(X_train))
print(sample_gsa(X_train).shape)

(54514, 32, 8)
(54514, 32, 8)


In [42]:
sample_encoder_layer = ParticleAttentionBlock(d_model = 8, num_heads=8, dff = 2048)

print(np.shape(X_train))
print(sample_encoder_layer(X_train).shape)

(54514, 32, 8)
(54514, 32, 8)


In [43]:
sample_encoder = Encoder(track_layers = track_layers, num_layers=4, d_model = 128, num_heads= 8, dff = 2048, MASKVAL=MASKVAL)
sample_encoder_output = sample_encoder(X_train)
print(X_train.shape)
print(sample_encoder_output.shape)

(54514, 32, 8)
(54514, 32, 128)


In [45]:
y_train_1 = np.reshape(y_train, (y_train.shape[0], y_train.shape[1],1 ))
print(y_train_1.shape)
y_valid_1 = np.reshape(y_valid, (y_valid.shape[0], y_valid.shape[1],1 ))
print(y_valid_1.shape)

(54514, 3, 1)
(13629, 3, 1)


In [46]:
sample_decoder = Decoder(num_layers= 4, d_model = 128, n_targets= 3, num_heads= 8, MASKVAL = MASKVAL, dff = 128)

output = sample_decoder(x = y_train_1, context = X_train)

print(X_train.shape)
print(y_train.shape)
print(output.shape)

(54514, 32, 8)
(54514, 3)
(54514, 128)


In [47]:
sample_jet_layer = Jet_Layer(n_targets=3,dff = 512, d_model=128,MASKVAL=-999)
output = sample_jet_layer(output)
print(output.shape)

(54514, 9)


In [48]:
track_layers = [32,32,32,32]
num_layers = 4
num_heads = 9
dff = 128
MASKVAL = -999
dropout = 0.1
n_targets = bhads.shape[1]
d_model = 8

In [49]:
sample_hffragTransformer = hffragTransformer(
    track_layers=track_layers,
    num_layers = num_layers,
    dff = dff,
    num_heads=num_heads,
    MASKVAL = MASKVAL,
    dropout= dropout,
    n_targets= n_targets,
    d_model = d_model
)

In [50]:
output = sample_hffragTransformer((X_train,y_train_1))

In [51]:
print(np.shape(X_train))
print(output.shape)

(54514, 32, 8)
(54514, 9)


In [52]:
sample_hffragTransformer.summary()

Model: "hffrag_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  23208     
                                                                 
 decoder_2 (Decoder)         multiple                  29210     
                                                                 
 jet__layer_1 (Jet_Layer)    multiple                  2265      
                                                                 
Total params: 54,683
Trainable params: 54,427
Non-trainable params: 256
_________________________________________________________________


In [53]:
sample_hffragTransformer.compile(
    optimizer=tf.keras.optimizers.Nadam(LR),
    loss = Mean_Squared_Error
)

In [54]:
history = sample_hffragTransformer.fit(
    (X_train,y_train_1), y_train,
    validation_data= ((X_valid,y_valid_1),y_valid),
    batch_size=BATCHSIZE,
    epochs=EPOCHS,
)

Epoch 1/100
426/426 [==============================] - 64s 76ms/step - loss: 47476047872.0000 - val_loss: 48103899136.0000
Epoch 2/100
426/426 [==============================] - 30s 69ms/step - loss: 47456686080.0000 - val_loss: 48099946496.0000
Epoch 3/100
156/426 [=========>....................] - ETA: 17s - loss: 47052754944.0000

KeyboardInterrupt: 